<a href="https://colab.research.google.com/github/tomheston/A-Comparison-of-the-Relative-Risk-Index-with-Unit-Fragilty/blob/main/UFI_vs_FI_vs_RRI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# Thomas F. Heston
# GNU GPL v3.0
# I am an academic, please give me a citation if you use this.
# https://faculty.washington.edu/theston

# Importing essential libraries for data handling and statistical analysis
import pandas as pd
from scipy.stats import fisher_exact, pearsonr
from scipy.stats import chi2_contingency
from scipy.stats import norm
import numpy as np

# set variables
lowran = 50 # lowest number in the 2x2 table
highestran = 150 # highest number in 2x2 table
toppv=0.05 # highest p-value
lowpv=0.000999 # lowest p-value
cases = 1000 # how many cases to run

# Function to generate data and compute p-values
def generate_data_and_pvalues(rows=cases):
    data = []
    # start loop
    while len(data) < rows:
        highran = np.random.randint(lowran+1, highestran)
        ao, bo, co, do = [int(x) for x in np.random.randint(lowran, highran, 4)]
        zero_found = False  # Add this flag at the beginning of the loop

        # we will use chi2 or fishers exact test to calculate p-values
        #_, pv, _, _ = chi2_contingency([[ao, bo], [co, do]],correction=True)
        _, pv = fisher_exact([[ao, bo], [co, do]])

        #
        # For this program, the UFI/FI is calculated only pv < 0.05
        # UFI: marginal totals are all kept equal.
        # FI: marginal totals are not kept equal.
        # For both: if p<0.05 then add to smallest cell otherwise subtract
        # from largest cell.
        #
        if lowpv < pv < toppv:
            #
            # A weakness of the UFI and FI is the confusion over which cell
            # to modify if more than one cell = min or max, so to simplify
            # this program we will only analyze cells with nonequal values.
            #
            if ao == bo or ao == co or ao == do or bo == co or bo == do or co == do:
              #break
              continue
            #
            smallest = min(ao, bo, co, do)
            #a1, b1, c1, d1 = ao, bo, co, do

            # Add retry logic
            retry_count = 0
            max_retries = 100

            # calculate the FI
            fi = 1
            pv1 = 0
            max_iter = 100  # or some reasonable limit
            count = 0
            while pv1 < toppv and count < max_iter and retry_count < max_retries:
                if smallest == ao:
                    a1, b1, c1, d1 = ao+fi, bo-fi, co, do
                elif smallest == bo:
                    a1, b1, c1, d1 = ao-fi, bo+fi, co, do
                elif smallest == co:
                    a1, b1, c1, d1 = ao, bo, co+fi, do-fi
                elif smallest == do:
                    a1, b1, c1, d1 = ao, bo, co-fi, do+fi
                #
                # if the smallest cell goes to zero then UFI/FI calculations
                # break, so throw out these cases
                if any(val == 0 for val in [a1, b1, c1, d1]):
                    zero_found = True  # Set the flag to True if zero found
                    #break
                    retry_count += 1
                    if retry_count > max_retries:
                        continue
                    smallest = min(ao, bo, co, do)
                    continue
                # calculate change in p-value (pv1)
                #_, pv1, _, _ = chi2_contingency([[a1, b1], [c1, d1]], correction=True)
                _, pv1 = fisher_exact([[a1, b1], [c1, d1]]) # pv= the p-value
                #
                if pv1 < toppv:
                    fi += 1
                count += 1
                retry_count += 1

            # Reset retry count
            retry_count = 0

            ufi=1
            pv2 = 0
            max_iter = 100  # or some reasonable limit
            count = 0
            while pv2 < toppv and count < max_iter and retry_count < max_retries:
                if smallest in (ao, do):
                    a2, b2, c2, d2 = ao+ufi, bo-ufi, co-ufi, do+ufi
                elif smallest in (bo, co):
                    a2, b2, c2, d2 = ao-ufi, bo+ufi, co+ufi, do-ufi
                #
                # if the smallest cell goes to zero then UFI/FI calculations
                # break, so throw out these cases
                if any(val == 0 for val in [a2, b2, c2, d2]):
                    zero_found = True  # Set the flag to True if zero found
                    #break
                    retry_count += 1
                    if retry_count > max_retries:
                        continue
                    smallest = min(ao, bo, co, do)
                    continue
                # calculate change in p-value (pv1)
                #_, pv2, _, _ = chi2_contingency([[a2, b2], [c2, d2]], correction=True)
                _, pv2 = fisher_exact([[a2, b2], [c2, d2]]) # pv= the p-value
                #
                if pv2 < toppv:
                    ufi += 1
                count += 1
                retry_count += 1

            if zero_found:  # Check the flag after the inner loop
              continue  # Continue the outer loop if zero found

            # RRI calculations
            ppv1 = ao / (ao + bo)
            ppv2 = co / (co + do)
            inc_ppv = 1
            ao2, bo2, co2, do2 = ao, bo, co, do
            total = ao + bo + co + do
            FQ1 = fi / total
            FQ2 = ufi / total
            RRI = abs((bo*co-ao*do)/(ao+bo+co+do)) # exact RR index
            if ppv1>ppv2:
              ppv3a=(ao2-RRI)/(ao2+bo2)
              ppv3b=(co2+RRI)/(co2+do2)
            elif ppv1<ppv2:
              ppv3a=(ao2+RRI)/(ao2+bo2)
              ppv3b=(co2-RRI)/(co2+do2)
            pRRI = RRI/(ao + bo + co + do) # the average percent change in ao, bo, co and do to get them to a RR of 1.
            data.append([ao, bo, co, do, pv, a1, b1, c1, d1, pv1, fi, FQ1, a2, b2, c2, d2, pv2, ufi, FQ2, ao2, bo2, co2, do2, ppv1, ppv2, ppv3a, ppv3b, RRI, pRRI])

    columns = ['ao', 'bo', 'co', 'do', 'pv', 'a1', 'b1', 'c1', 'd1', 'pv1', 'fi', 'FQ1', 'a2', 'b2', 'c2', 'd2', 'pv2', 'ufi', 'FQ2', 'ao2', 'bo2', 'co2', 'do2', 'ppv1', 'ppv2', 'ppv3a', 'ppv3b', 'RRI','pRRI']
    df = pd.DataFrame(data, columns=columns)
    df['size'] = df['ao'] + df['bo'] + df['co'] + df['do']
    df['pv'] = df['pv'].round(7)
    df['pv1'] = df['pv1'].round(7)

    # Correlated with p-value
    corr_fi_pv, p_value_fi_pv = [round(val, 5) for val in pearsonr(df['fi'], df['pv'])]
    corr_FQ1_pv, p_value_FQ1_pv = [round(val, 5) for val in pearsonr(df['FQ1'], df['pv'])]
    corr_ufi_pv, p_value_ufi_pv = [round(val, 5) for val in pearsonr(df['ufi'], df['pv'])]
    corr_FQ2_pv, p_value_FQ2_pv = [round(val, 5) for val in pearsonr(df['FQ2'], df['pv'])]
    corr_RRI_pv, p_value_RRI_pv = [round(val, 5) for val in pearsonr(df['RRI'], df['pv'])]
    corr_pRRI_pv, p_value_pRRI_pv = [round(val, 5) for val in pearsonr(df['pRRI'], df['pv'])]

    #corr_fi_RRI, p_value_fi_RRI = [round(val, 5) for val in pearsonr(df['fi'], df['RRI'])]
    #corr_FQ1_RRI, p_value_FQ1_RRI = [round(val, 5) for val in pearsonr(df['FQ1'], df['RRI'])]
    #corr_ufi_RRI, p_value_ufi_RRI = [round(val, 5) for val in pearsonr(df['ufi'], df['RRI'])]
    #corr_FQ2_RRI, p_value_FQ2_RRI = [round(val, 5) for val in pearsonr(df['FQ2'], df['RRI'])]

    #corr_fi_pRRI, p_value_fi_pRRI = [round(val, 5) for val in pearsonr(df['fi'], df['pRRI'])]
    #corr_FQ1_pRRI, p_value_FQ1_pRRI = [round(val, 5) for val in pearsonr(df['FQ1'], df['pRRI'])]
    #corr_ufi_pRRI, p_value_ufi_pRRI = [round(val, 5) for val in pearsonr(df['ufi'], df['pRRI'])]
    #corr_FQ2_pRRI, p_value_FQ2_pRRI = [round(val, 5) for val in pearsonr(df['FQ2'], df['pRRI'])]


    # Calculate the averages
    avgfi = df['fi'].mean()
    avgufi = df['ufi'].mean()
    avgFQ1 = df['FQ1'].mean()
    avgFQ2 = df['FQ2'].mean()
    avgRRI = df['RRI'].mean()
    avgpRRI = df['pRRI'].mean()

    # Print out base settings
    print("BASE SETTINGS")
    print("Total Cases Tested:", cases)
    print("Lowest number in 2x2 table:", lowran)
    print("Highest number in 2x2 table:", highestran)
    print("Lowest p-value:", lowpv)
    print("Highest p-value:", toppv)
    print()

    # Print out correlation results with p-values
    print("CORRELATIONS")
    print("Correlation between FI and pv:", corr_fi_pv, "avg:", avgfi)
    print("Correlation between UFI and pv:", corr_ufi_pv, "avg:", avgufi)
    print("Correlation between FQ1 and pv:", corr_FQ1_pv, "avg:", avgFQ1)
    print("Correlation between FQ2 and pv:", corr_FQ2_pv, "avg:", avgFQ2)
    print("Correlation between RRI and pv:", corr_RRI_pv, "avg:", avgRRI)
    print("Correlation between pRRI and pv:", corr_pRRI_pv, "avg:", avgpRRI)
    print("------")
    # Correlation coefficients
    r1 = corr_fi_pv
    r2 = corr_pRRI_pv
    # Sample sizes
    n1 = len(df['fi'])
    n2 = len(df['FQ1'])
    # Fisher's Z transform
    z1 = 0.5*np.log((1+r1)/(1-r1))
    z2 = 0.5*np.log((1+r2)/(1-r2))
    # Standard errors
    se1 = 1/np.sqrt(n1-3)
    se2 = 1/np.sqrt(n2-3)
    # Z-statistic
    z = (z1 - z2) / np.sqrt(se1**2 + se2**2)
    # p-value
    p = 2*norm.cdf(-abs(z))
    # Interpret
    alpha = 0.05
    if p < alpha:
      print("Correlations are significantly different")
    else:
      print("Correlations are not significantly different")

    #print("Correlation between FI and RRI:", corr_fi_RRI)
    #print("Correlation between UFI and RRI:", corr_ufi_RRI)
    #print("Correlation between FQ1 and RRI:", corr_FQ1_RRI)
    #print("Correlation between FQ2 and RRI:", corr_FQ2_RRI)
    #print("------")
    #print("Correlation between FI and pRRI:", corr_fi_pRRI)
    #print("Correlation between UFI and pRRI:", corr_ufi_pRRI)
    #print("Correlation between FQ1 and pRRI:", corr_FQ1_pRRI)
    #print("Correlation between FQ2 and pRRI:", corr_FQ2_pRRI)
    #print()

    return df

# Generate the DataFrame
df = generate_data_and_pvalues()

# Print the DataFrame to a file (optional)
# and Export from Colab (optional)
df.to_csv('2x2_tables.csv', index=False)
from google.colab import files
files.download('2x2_tables.csv')


BASE SETTINGS
Total Cases Tested: 250
Lowest number in 2x2 table: 50
Highest number in 2x2 table: 150
Lowest p-value: 0.000999
Highest p-value: 0.05

CORRELATIONS
Correlation between FI and pv: -0.90364 avg: 5.28
Correlation between UFI and pv: -0.88881 avg: 3.036
Correlation between FQ1 and pv: -0.88755 avg: 0.015668408079152936
Correlation between FQ2 and pv: -0.87791 avg: 0.009008227868135887
Correlation between RRI and pv: -0.83787 avg: 11.668886175350364
Correlation between pRRI and pv: -0.76455 avg: 0.034809533797610086
------
Correlations are significantly different


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>